In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Import the datetime module from the datetime library.
from datetime import datetime

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Use the tuple() function to display the latitude and longitude combinations.
for coordinate in coordinates:
    coordinate = (coordinate[0], coordinate[1])

In [5]:
from citipy import citipy
for coordinate in coordinates:
    city_name = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    city_countryCode = citipy.nearest_city(coordinate[0], coordinate[1]).country_code

In [6]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)
#print(cities)

620

In [7]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [8]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

In [9]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [10]:
# Loop through all the cities in our list.
for i in range(len(cities)):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + cities[i]

In [11]:
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    #print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        #print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

-----------------------------
Data Retrieval Complete      
-----------------------------


In [12]:
len(city_data)

571

In [13]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Vaini,-21.2000,-175.2000,71.76,88,75,13.80,TO,2021-07-14 04:17:00
1,Vanino,49.0869,140.2543,84.72,55,97,7.92,RU,2021-07-14 04:21:54
2,Hobart,-42.8794,147.3294,55.11,90,75,1.01,AU,2021-07-14 04:14:45
3,Guerrero Negro,27.9769,-114.0611,69.91,80,21,13.82,MX,2021-07-14 04:15:40
4,Bluff,-46.6000,168.3333,49.80,80,100,22.84,NZ,2021-07-14 04:16:42
5,Alofi,-19.0595,-169.9187,78.69,89,50,11.50,NU,2021-07-14 04:17:33
6,Saldanha,-33.0117,17.9442,48.54,71,38,6.13,ZA,2021-07-14 04:18:57
7,Goderich,43.7501,-81.7165,67.91,94,98,8.05,CA,2021-07-14 04:18:03
8,Punta Arenas,-53.1500,-70.9167,28.51,93,75,3.44,CL,2021-07-14 04:16:42
9,Voh,-20.9667,164.7000,74.71,46,0,8.79,NC,2021-07-14 04:21:55


In [14]:
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Vaini,TO,2021-07-14 04:17:00,-21.2000,-175.2000,71.76,88,75,13.80
1,Vanino,RU,2021-07-14 04:21:54,49.0869,140.2543,84.72,55,97,7.92
2,Hobart,AU,2021-07-14 04:14:45,-42.8794,147.3294,55.11,90,75,1.01
3,Guerrero Negro,MX,2021-07-14 04:15:40,27.9769,-114.0611,69.91,80,21,13.82
4,Bluff,NZ,2021-07-14 04:16:42,-46.6000,168.3333,49.80,80,100,22.84
5,Alofi,NU,2021-07-14 04:17:33,-19.0595,-169.9187,78.69,89,50,11.50
6,Saldanha,ZA,2021-07-14 04:18:57,-33.0117,17.9442,48.54,71,38,6.13
7,Goderich,CA,2021-07-14 04:18:03,43.7501,-81.7165,67.91,94,98,8.05
8,Punta Arenas,CL,2021-07-14 04:16:42,-53.1500,-70.9167,28.51,93,75,3.44
9,Voh,NC,2021-07-14 04:21:55,-20.9667,164.7000,74.71,46,0,8.79


In [15]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")